In [1]:
import os
import pandas as pd
import numpy as np

import rasterio
import sample_rasters as sr
import planetary_computer as pc
import geopandas as gpd

from scipy.ndimage import maximum_filter as maxf2D
from scipy.ndimage import minimum_filter as minf2D
from scipy.ndimage import convolve as conf2D

from shapely.geometry import Point
from rasterio.crs import CRS

In [2]:
# *********************************************************************

def min_raster(rast_reader, band, rast_name, n, folder_path=''):  
    """
        Creates a new raster by replacing each pixel p in given raster R by the minimum value in a nxn window centered at p.
        The raster with minimum values is saved in a temp folder in the current working directory if no folder_path is given.
            Parameters: 
                        rast_reader (rasterio.io.DatasetReader):
                            reader to the raster from which to compute the minimum values in a window
                        rast_name (str):
                            name of raster. The resulting raster will be saved as rast_name_maxs.tif.
                        n (int):
                            Side length (in pixels) of the square window over which to compute minimum values for each pixel.
                        folder_path (str):
                            directory where to save raster. If none is given, then it saves the raster in a temp folder in the cwd.
            Return: None    
    """
    rast = rast_reader.read([band]).squeeze() # read raster values
    mins = minf2D(rast, size=(n,n))    # calculate min in window
    
    if not folder_path:                         # if needed, create temp directory to save files 
        folder_path = make_directory('temp')
    
    dtype = rasterio.dtypes.get_minimum_dtype(mins)  # parameters for saving
    
    fp = os.path.join(folder_path, rast_name +'_mins.tif')      # save raster
    sr.save_raster(mins, 
                fp, 
                rast.shape,
                1,
                rast_reader.crs, 
                rast_reader.transform, 
                dtype)  
    return

# ------------------------------------------------------------------------------

def max_raster(rast_reader, band, rast_name, n, folder_path=''):  
    """
        Creates a new raster by replacing each pixel p in given raster R by the max value in a nxn window centered at p.
        The raster with maximum values is saved in a temp folder in the current working directory if no folder_path is given.
            Parameters: 
                        rast_reader (rasterio.io.DatasetReader):
                            reader to the raster from which to compute the maximum values in a window
                        rast_name (str):
                            name of raster. The resulting raster will be saved as rast_name_maxs.tif.
                        n (int):
                            Side length (in pixels) of the square window over which to compute maximum values for each pixel.
                        folder_path (str):
                            directory where to save raster. If none is given, then it saves the raster in a temp folder in the cwd.
            Return: None    
    """
    rast = rast_reader.read([band]).squeeze() # read raster values
    maxs = maxf2D(rast, size=(n,n))    # calculate min in window
    
    if not folder_path:                         # if needed, create temp directory to save files 
        folder_path = make_directory('temp')
    
    dtype = rasterio.dtypes.get_minimum_dtype(maxs)  # parameters for saving
    
    fp = os.path.join(folder_path, rast_name +'_maxs.tif')      # save raster
    sr.save_raster(maxs, 
                fp, 
                rast.shape,
                1,
                rast_reader.crs, 
                rast_reader.transform, 
                dtype)  
    return

# ------------------------------------------------------------------------------

def avg_raster(rast_reader, band, rast_name, n, folder_path=''): 
    """
        Creates a new raster by replacing each pixel p in given raster R by the avg value in a nxn window centered at p.
        The raster with averege values is saved in a temp folder in the current working directory if no folder_path is given.
            Parameters: 
                        rast_reader (rasterio.io.DatasetReader):
                            reader to the raster from which to compute the average values in a window
                        rast_name (str):
                            name of raster. The resulting raster will be saved as rast_name_avgs.tif.
                        n (int):
                            Side length (in pixels) of the square window over which to compute average values for each pixel.
                        folder_path (str):
                            directory where to save raster. If none is given, then it saves the raster in a temp folder in the cwd.
            Return: None    
    """
    rast = rast_reader.read([band]).squeeze() # read raster values

    w = np.ones(n*n).reshape(n,n)      # calculate averages in window
    avgs = conf2D(rast, 
             weights=w,
             mode='constant')
    avgs = avgs/(n*n)
    
    # if needed, create temp directory to save files 
    if not folder_path:  
        folder_path = make_directory('temp')
            
    # parameters for saving   
    fp = os.path.join(folder_path, rast_name +'_avgs.tif')                
    dtype = rasterio.dtypes.get_minimum_dtype(avgs)
            
    sr.save_raster(avgs,    # save rasters
                fp, 
                rast.shape, 
                1,
                rast_reader.crs, 
                rast_reader.transform, 
                dtype)  
    return

In [3]:
# ***************************************************
# ************* NOTEBOOK VARIABLES ******************

itemids = ['ca_m_3411934_sw_11_.6_20160713_20161004']
itemid = 'ca_m_3411934_sw_11_.6_20160713_20161004'
# open csv with all itemids

all_pts = pd.read_csv(os.path.join(os.getcwd(), 'test_set.csv'))
all_pts.head()
# add train set

# ***************************************************
# ***************************************************

,x,y,pts_crs,aoi,naip_id,polygon_id,r,g,b,nir,ndvi,year,month,day_in_year,lidar,max_lidar,min_lidar,min_max_diff,avg_lidar,iceplant
0,237982.367763,3.811483e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,14,81,78,66,159,0.325000,2016,7,195,2.0,12.0,0.0,12.0,3.333333,0.0
1,240930.697752,3.812113e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,45,61,62,57,155,0.435185,2016,7,195,1.0,1.0,0.0,1.0,0.333333,1.0
2,238527.146788,3.810765e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,0,43,41,46,112,0.445161,2016,7,195,0.0,0.0,0.0,0.0,0.000000,1.0
3,239461.437263,3.812015e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,7,68,60,58,145,0.361502,2016,7,195,2.0,6.0,1.0,5.0,2.888889,1.0
4,239158.424887,3.812089e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,24,73,65,65,102,0.165714,2016,7,195,0.0,0.0,0.0,0.0,0.000000,0.0


In [4]:
pts_list = []
#for itemid in itemids:
 

In [5]:
# ------------------------------
# Filter points in test set and train set with this itemid
pts = all_pts.loc[all_pts['naip_id'] == itemid]
pts.head()

,x,y,pts_crs,aoi,naip_id,polygon_id,r,g,b,nir,ndvi,year,month,day_in_year,lidar,max_lidar,min_lidar,min_max_diff,avg_lidar,iceplant
0,237982.367763,3.811483e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,14,81,78,66,159,0.325000,2016,7,195,2.0,12.0,0.0,12.0,3.333333,0.0
1,240930.697752,3.812113e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,45,61,62,57,155,0.435185,2016,7,195,1.0,1.0,0.0,1.0,0.333333,1.0
2,238527.146788,3.810765e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,0,43,41,46,112,0.445161,2016,7,195,0.0,0.0,0.0,0.0,0.000000,1.0
3,239461.437263,3.812015e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,7,68,60,58,145,0.361502,2016,7,195,2.0,6.0,1.0,5.0,2.888889,1.0
4,239158.424887,3.812089e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,24,73,65,65,102,0.165714,2016,7,195,0.0,0.0,0.0,0.0,0.000000,0.0


In [6]:
# ------------------------------
# Open NAIP scene and calculate auxiliary spectral rasters
item = sr.get_item_from_id(itemid)    # locate raster
href = pc.sign(item.assets["image"].href)
lidar_rast_r = rasterio.open(href)

In [7]:
folp = os.path.join(os.getcwd(),'temp')

rast_name = 'r_'+itemid 
min_raster(rast_reader = lidar_rast_r, band=1, rast_name = rast_name, n=3, folder_path=folp)
max_raster(rast_reader = lidar_rast_r, band=1,  rast_name = rast_name, n=3, folder_path=folp)
avg_raster(rast_reader = lidar_rast_r, band=1, rast_name = rast_name, n=3, folder_path=folp)
print('finished R aux rasters')

rast_name = 'g_'+itemid 
min_raster(rast_reader = lidar_rast_r, band=2, rast_name = rast_name, n=3, folder_path=folp)
max_raster(rast_reader = lidar_rast_r, band=2,  rast_name = rast_name, n=3, folder_path=folp)
avg_raster(rast_reader = lidar_rast_r, band=2, rast_name = rast_name, n=3, folder_path=folp)
print('finished G aux rasters')    

rast_name = 'b_'+itemid 
min_raster(rast_reader = lidar_rast_r, band=3, rast_name = rast_name, n=3, folder_path=folp)
max_raster(rast_reader = lidar_rast_r, band=3,  rast_name = rast_name, n=3, folder_path=folp)
avg_raster(rast_reader = lidar_rast_r, band=3, rast_name = rast_name, n=3, folder_path=folp)
print('finished B aux rasters')    

rast_name = 'nir_'+itemid 
min_raster(rast_reader = lidar_rast_r, band=4, rast_name = rast_name, n=3, folder_path=folp)
max_raster(rast_reader = lidar_rast_r, band=4,  rast_name = rast_name, n=3, folder_path=folp)
avg_raster(rast_reader = lidar_rast_r, band=4, rast_name = rast_name, n=3, folder_path=folp)
print('finished NIR aux rasters')    

finished R aux rasters
finished G aux rasters
finished B aux rasters
finished NIR aux rasters


In [8]:
# ------------------------------
# Convert csv to geopandas

## Get points information from csv
crs = CRS.from_string(pts.pts_crs[0])
#TO DO: change geodataframe_from_csv to use pandas df and not open file there
#    pts = sr.geodataframe_from_csv(pts_fp, 'x','y',crs)
if 'geometry' in pts.columns:           # rename geometry column if it exists
        pts = pts.rename(columns = {'geometry': 'geometry_0'})

# recreate geometry column as shapely Points
xy = []
for x,y in zip(pts['x'], pts['y']):
    xy.append(Point(x,y))

pts_col = gpd.GeoDataFrame(pd.DataFrame(xy, columns=['geometry']), crs=crs)
pts_col = pts_col.to_crs(lidar_rast_r.crs).geometry
pts_col

/srv/conda/envs/notebook/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


0       POINT (237982.368 3811482.620)
1       POINT (240930.698 3812112.671)
2       POINT (238527.147 3810765.439)
3       POINT (239461.437 3812015.077)
4       POINT (239158.425 3812089.409)
                     ...              
7615    POINT (235739.565 3812116.151)
7616    POINT (240907.098 3812103.126)
7617    POINT (238234.510 3810942.780)
7618    POINT (235932.921 3811830.180)
7619    POINT (240920.422 3812100.323)
Name: geometry, Length: 7620, dtype: geometry

In [9]:
# ------------------------------
## Sample canopy_height at point, and max, min and avg canopy height around point

samples = []
for band in ['r_','g_','b_','nir_']:
    for tag in ['_maxs', '_mins', '_avgs']:

        fp = os.path.join(os.getcwd(), 'temp', band + itemid + tag + '.tif')
        col_name = band.replace('_', '') + tag.replace('s','')

        rast_r = rasterio.open(fp)
        sample = sr.sample_raster_from_pts(pts_col, rast_r, [col_name])

        samples.append(sample)

new_features = pd.concat(samples, axis = 1)
new_features

,r_max,r_min,r_avg,g_max,g_min,g_avg,b_max,b_min,b_avg,nir_max,nir_min,nir_avg
0,97,52,20.444445,89,65,21.555555,73,55,8.111111,170,151,16.444445
1,70,61,7.888889,69,62,9.111111,59,57,1.222222,162,155,15.333333
2,43,40,13.000000,42,39,12.222222,48,45,18.555555,112,100,23.666666
3,70,66,10.888889,63,58,3.888889,60,56,0.888889,151,143,4.222222
4,74,64,14.000000,66,62,7.111111,67,64,8.666667,111,95,15.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
7615,78,71,17.777779,70,65,10.222222,72,69,13.111111,112,98,18.555555
7616,71,56,3.000000,63,55,1.000000,65,57,1.777778,157,137,3.777778
7617,43,38,11.555555,43,38,11.555555,49,44,17.666666,113,81,18.444445
7618,66,61,7.555555,63,59,5.111111,61,58,2.444444,151,139,3.222222


In [10]:
# ------------------------------
## Add all derived spectral data to pts dataframe

pts = pd.concat([pts, new_features], axis=1)
pts

,x,y,pts_crs,aoi,naip_id,polygon_id,r,g,b,nir,...,r_avg,g_max,g_min,g_avg,b_max,b_min,b_avg,nir_max,nir_min,nir_avg
0,237982.367763,3.811483e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,14,81,78,66,159,...,20.444445,89,65,21.555555,73,55,8.111111,170,151,16.444445
1,240930.697752,3.812113e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,45,61,62,57,155,...,7.888889,69,62,9.111111,59,57,1.222222,162,155,15.333333
2,238527.146788,3.810765e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,0,43,41,46,112,...,13.000000,42,39,12.222222,48,45,18.555555,112,100,23.666666
3,239461.437263,3.812015e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,7,68,60,58,145,...,10.888889,63,58,3.888889,60,56,0.888889,151,143,4.222222
4,239158.424887,3.812089e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,24,73,65,65,102,...,14.000000,66,62,7.111111,67,64,8.666667,111,95,15.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7615,235739.564561,3.812116e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,49,78,69,70,112,...,17.777779,70,65,10.222222,72,69,13.111111,112,98,18.555555
7616,240907.098289,3.812103e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,47,60,57,58,137,...,3.000000,63,55,1.000000,65,57,1.777778,157,137,3.777778
7617,238234.510151,3.810943e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,4,38,39,45,107,...,11.555555,43,38,11.555555,49,44,17.666666,113,81,18.444445
7618,235932.920829,3.811830e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,10,65,63,59,149,...,7.555555,63,59,5.111111,61,58,2.444444,151,139,3.222222


In [11]:
for band in ['r_','g_','b_','nir_']:
    col_name = band + 'diff'
    pts[col_name] = pts[band +'max'] - pts[band +'min']
pts

,x,y,pts_crs,aoi,naip_id,polygon_id,r,g,b,nir,...,b_max,b_min,b_avg,nir_max,nir_min,nir_avg,r_diff,g_diff,b_diff,nir_diff
0,237982.367763,3.811483e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,14,81,78,66,159,...,73,55,8.111111,170,151,16.444445,45,24,18,19
1,240930.697752,3.812113e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,45,61,62,57,155,...,59,57,1.222222,162,155,15.333333,9,7,2,7
2,238527.146788,3.810765e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,0,43,41,46,112,...,48,45,18.555555,112,100,23.666666,3,3,3,12
3,239461.437263,3.812015e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,7,68,60,58,145,...,60,56,0.888889,151,143,4.222222,4,5,4,8
4,239158.424887,3.812089e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,24,73,65,65,102,...,67,64,8.666667,111,95,15.666667,10,4,3,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7615,235739.564561,3.812116e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,49,78,69,70,112,...,72,69,13.111111,112,98,18.555555,7,5,3,14
7616,240907.098289,3.812103e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,47,60,57,58,137,...,65,57,1.777778,157,137,3.777778,15,8,8,20
7617,238234.510151,3.810943e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,4,38,39,45,107,...,49,44,17.666666,113,81,18.444445,5,5,5,32
7618,235932.920829,3.811830e+06,epsg:26911,campus_lagoon,ca_m_3411934_sw_11_.6_20160713_20161004,10,65,63,59,149,...,61,58,2.444444,151,139,3.222222,5,4,3,12


In [12]:
# ------------------------------    
# Clean dataframe
#pts.drop(['geometry'],axis=1, inplace=True) # remove geometry column (already have lat,lon and CRS)
pts = pts[['x', 'y', 'pts_crs', #  point location
         'aoi', 'naip_id', 'polygon_id',  # sampling info
         'r', 'r_max', 'r_min', 'r_diff', 'r_avg',
         'g', 'g_max', 'g_min', 'g_diff', 'g_avg',
         'b', 'b_max', 'b_min', 'b_diff', 'b_avg',
         'nir', 'nir_max', 'nir_min', 'nir_diff', 'nir_avg',
         'ndvi',     # spectral
         'year', 'month', 'day_in_year', # date
         'lidar', 'max_lidar', 'min_lidar', 'min_max_diff', 'avg_lidar', # lidar
         'iceplant'
         ]] 
pts.columns

Index(['x', 'y', 'pts_crs', 'aoi', 'naip_id', 'polygon_id', 'r', 'r_max',
       'r_min', 'r_diff', 'r_avg', 'g', 'g_max', 'g_min', 'g_diff', 'g_avg',
       'b', 'b_max', 'b_min', 'b_diff', 'b_avg', 'nir', 'nir_max', 'nir_min',
       'nir_diff', 'nir_avg', 'ndvi', 'year', 'month', 'day_in_year', 'lidar',
       'max_lidar', 'min_lidar', 'min_max_diff', 'avg_lidar', 'iceplant'],
      dtype='object')

In [16]:
# ------------------------------
# Delete auxiliary LIDAR rasters created for this year
for band in ['r_','g_','b_','nir_']:
    for tag in ['_maxs', '_mins', '_avgs']:
        fp = os.path.join(os.getcwd(), 'temp', band + itemid + tag + '.tif')   
        os.remove(fp)

In [17]:
pts_list.append(pts)

all_pts = pd.concat(pts_list, axis =0)
all_pts

In [ ]:
# ------------------------------
## Save points with added spectral data
ptslidar_fp = os.path.join(os.getcwd(), 
                           'temp', 
                           aoi +'_pts_spectral_lidar_'+str(year)+'.csv')
pts.to_csv(ptslidar_fp, index=False)

# ------------------------------
## Delete original csv files (points without LIDAR)
if delete_pts == True:
    os.remove(pts_fp)